# 実際のWebページに対するスクレイピングのサンプル

以下は，いずれも動的Webページに対するサンプルですが，抽出部の指定については静的・動的に関わらず参考になるかもしれません．

## 準備

### ライブラリのインストール

In [1]:
!pip install playwright beautifulsoup4 lxml_html_clean
!playwright install
!playwright install-deps

Installing dependencies...
Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entr

### htmlデータ取得用の関数定義

In [2]:
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def fetch_html(url):
    # Playwrightの非同期APIを用いてHTMLを取得
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # ページを取得（ネットワークが落ち着くまで待つ）
        await page.goto(url, wait_until="networkidle")

        # レンダリング後のHTML文字列を取得
        html = await page.content()

        await browser.close()

    # レンダリング済みHTML文字列を返す
    return html

## Yahoo リアルタイム検索のトレンドの抽出

In [3]:
# レンダリング済みHTMLの取得
html = await fetch_html("https://search.yahoo.co.jp/realtime")

# レンダリング結果をBeautifulSoupで解析
soup = BeautifulSoup(html, 'html.parser')

# トレンド項目の抽出
trend_items = soup.select('.Trend_container__d7dWI h1')

# 抽出結果を表示
print("Yahoo!リアルタイム検索のトレンド項目:")
for i, item in enumerate(trend_items, start=1):
    print("{:2}. {}".format(i, item.text.strip()))



Yahoo!リアルタイム検索のトレンド項目:
 1. デビュー6周年
 2. 篠塚の学習記録
 3. 漣ジュン
 4. 楽しかったですか
 5. アクア・ドラゴン
 6. 横山くん
 7. 読売新聞 世論調査
 8. しゅんでる
 9. 小野友樹
10. 読んだら明日幸せになる
11. ヒメヒナバブライブ
12. 山本和臣
13. 久保ってる
14. 鎮西寿々歌
15. 相続馬限定馬主
16. 増田俊樹
17. UNDEAD
18. 山王耕造
19. 愛してくれて
20. 羽多野渉


## F1の2025 Driver Standingsの抽出

In [4]:
import pandas as pd

# レンダリング済みHTMLの取得
html = await fetch_html("https://www.formula1.com/en/results/2025/drivers")

# レンダリング結果をBeautifulSoupで解析
soup = BeautifulSoup(html, 'html.parser')

# 表の抽出
standings_table = soup.find('table', class_='Table-module_table__cKsW2')

# 表データの分割・DataFrameとして保存・表示
headers = [th.text.strip() for th in standings_table.find_all('th')]
rows = standings_table.find_all('tr')[1:]
data = []
for row in rows:
    cols = [td.text.strip() for td in row.find_all('td')]
    data.append(cols)
df = pd.DataFrame(data, columns=headers)
print(df)

   Pos.                Driver Nationality             Team Pts.
0     1       Lando NorrisNOR         GBR          McLaren  390
1     2      Oscar PiastriPIA         AUS          McLaren  366
2     3     Max VerstappenVER         NED  Red Bull Racing  366
3     4     George RussellRUS         GBR         Mercedes  294
4     5    Charles LeclercLEC         MON          Ferrari  226
5     6     Lewis HamiltonHAM         GBR          Ferrari  152
6     7     Kimi AntonelliANT         ITA         Mercedes  137
7     8    Alexander AlbonALB         THA         Williams   73
8     9       Isack HadjarHAD         FRA     Racing Bulls   51
9    10    Nico HulkenbergHUL         GER      Kick Sauber   49
10   11       Carlos SainzSAI         ESP         Williams   48
11   12     Oliver BearmanBEA         GBR     Haas F1 Team   41
12   13    Fernando AlonsoALO         ESP     Aston Martin   40
13   14        Liam LawsonLAW         NZL     Racing Bulls   36
14   15       Esteban OconOCO         FR